In [19]:
from konlpy.tag import Okt
import nltk
import re
import pickle
import os
path = os.getcwd()
with open('reviewSelected2500.p', 'rb') as f:
    selected_words = pickle.load(f)

last_table = []
okt = Okt()

def modify(text):
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)' #한글 자모음 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    pattern = '[^\w\s]' #특수기호 제거
    text = re.sub(pattern=pattern, repl=' ', string=text)
    return text

def tokenize(doc):
    return ['/'.join(t) for t in okt.pos(doc,norm=True,stem=True)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

def keyword_predict(s):
    model_best = models.load_model('ReviewModel')
    test_t = tokenize(s)
    tf = term_frequency(test_t)
    tf_sum = 0
    for i in tf:
        tf_sum = tf_sum + i
    if tf_sum == 0:
        tf_sum = 1
    final_tf = [i/tf_sum for i in tf]
    data = np.expand_dims(np.asarray(final_tf).astype('float32'),axis=0)
    score = float(model_best.predict(data))
    #print(score)

    if(score>0.55):
        #print("[{}]는 {:.2f}%확률로 만족 리뷰입니다.".format(s,score*100))
        return score
    elif(score<0.45):
        #print("[{}]는 {:.2f}%확률로 불만족 리뷰입니다.".format(s,(1-score)*100))
        return -(1-score)
    else:
        #print('그저 그렇다')
        return 0

In [21]:
import csv
import os
import numpy as np
from tensorflow.keras import models
score = {'가성비':0,'as':0,'배송':0,'성능':0} #총점
os.chdir('/Users/leesangyun/Documents/python/공모전')
f = open('LG전자 2020 그램15 15ZD995-VX50K (SSD 256GB).csv','r',encoding='utf-8-sig')
r = csv.reader(f)
os.chdir('/Users/leesangyun/Documents/python/modeling')
keyword = []
new_file = []
for i in score:
    keyword.append(i)

number = 0
for i in r:
    each_score = {'가성비':0,'as':0,'배송':0,'성능':0} #개별리뷰별 최상위 키워드에 대한 점수
    empty = []
    empty.append(i[7])
    dic = {} #해당 리뷰 내 최상위 키워드의 인덱스 넣는부분
    order = [] #단일 리뷰 내에서 출현한 최상위키워드를 순서대로 넣음
    number = number + 1
    for j in keyword: #키워드 첫 index 찾는부분
        num = i[7].find(j)
        dic[j] = num
    for k in dic: #index가 -1이 아니면 order list에 넣음
        if dic[k] !=-1:
            order.append(k)
            
    for l in range(len(order)-1): #리뷰에 최상위 키워드가 출현한 순서로 키워드리스트list 재배열
        for m in range(l+1,len(order)):
            if dic[order[l]]>dic[order[m]]:
                temp = order[l]
                order[l] = order[m]
                order[m] = temp
    for n in range(len(order)): #찾은 최상위 키워드의 인덱스를 사용하여 다음 최상위 키워드 전까지 슬라이싱하고 Predict
        if n != len(order)-1:
            line = i[7][dic[order[n]]:dic[order[n+1]]]
            each_score[order[n]] = keyword_predict(line)
            score[order[n]] = score[order[n]] + keyword_predict(line)
        else:
            line = i[7][dic[order[n]]:]
            each_score[order[n]] = keyword_predict(line)
            score[order[n]] = score[order[n]] + keyword_predict(line)
    for o in each_score: 
        empty.append(each_score[o])
        
    new_file.append(empty)
    
    
f.close()
print(new_file)



[['물건 받고 10일만에 액정 나가네요. 처음부터 하자 있는 물건이었다고 저는 생각합니다. 믿으실지 모르겠지만 액정보호필름 붙이고 나니 액정이 나갔는데 판매처도 LG도 그 누구도 책임지지 않습니다. 처음에 꼼꼼히 검수하지 않은 제 불찰도 있습니다만... 패널 금간 것을 일반인이 어떻게 확인을 해 낼 수 있을지...(액정에 기스를 보기는 했지만 교환하는데 시간 걸리고 귀찮으니 이정도는 감수하자 했는데, 그 기스가 액정기스가 아니라 lcd패널에 금이 간 것이었네요) 노트북을 떨어트리거나 찍히거나 한 적이 전혀 없어 자체의 문제라 생각합니다. 액정보호필름 붙이다 액정이 나가는 것을 지금까지 본 적도 들은 적도 없어서 매우 당황스럽습니다. 인터넷에 이런 경우가 있는지 알아보니 LG그램 자체가 액정 내구성이 엄청 안좋다고 합니다. - 손가락으로 액정을 잡았더니 액정이 나갔다 - 노트북을 닫았더니 액정이 나갔다 - 헝겊으로 닦다가 액정이 나갔다 등등 (13, 14년도에 적혀있던 글을 본 것이고 헝겊이야기는 들은 이야기 입니다.) 엘지그램 이쁘고 가볍고 좋습니다. 전시용으로 딱 좋습니다. 딱 거기까지 입니다. 이왕 구매하시는 거 무게가 좀 나가더라도 더 튼튼한 제품이 좋아 보입니다.', 0, 0, 0, 0], ['우선 가볍고, 디자인이 예뻐요.카페 등 외부 이동이 많으면 추천합니다.그런데, 뽑기를 잘못했는지, a자판이 다른 키보드와 누름는 키감이 다르네요.서비스센터 가라고 안내받았는데, 시간이 없어서 못가고있어요.눌러지긴 하는데, 눌르는 감이 없어서 눌렀나 계속 확인하게된다는,....처음부터 잘 만들어주시지, 바쁘게ㅠ암튼 휴대성은 짱입니다.', 0, 0, 0, 0], ['그램이 좋아서 샀는데 2in1이 너무 예뻐보이네여ㅠㅠ 그램도 좋은데 딴것도 둘러보세요', 0, 0, 0, 0], ['처음부터 문제가 있어서 많이 당황했습니다.지금은 서비스센터 갔다와서 괜찮지만 불안합니다.', 0, 0, 0, 0], ['배송이 생각보다 알맞게 도착했어요 좋은 가격에 잘 산거같어 기분이 좋

"\nlast_score = ['LG전자 2019 울트라PC']\nfor j in score:\n    sco = score[j]/number\n    last_score.append(sco)\n\nlast_table.append(last_score)"

In [22]:
last_score = ['LG전자 2020 그램15 15ZD995-VX50K (SSD 256GB)']
for j in score:
    sco = score[j]/number
    last_score.append(sco)

last_table.append(last_score)

last_table

[['LG전자 2020 그램15 15ZD995-VX50K (SSD 256GB)',
  0.012771530068197916,
  0.0017447052999984387,
  0.20539009037405945,
  0.0717325322849806]]

In [ ]:
final = open('last_table.csv','w',encoding='utf-8-sig')
w = csv.writer(final)

for i in last_table:
    w.writerow(i)
    
final.close()